In [3]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58f9a000 @  0x7f02300352a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
import torch

In [235]:
cuda = torch.cuda.is_available()
print(cuda)

True


In [0]:
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as dsets
from torch.autograd import Variable
import numpy as np
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [237]:
torch.manual_seed(10)
if cuda:
  torch.cuda.manual_seed(10)
'''
LOADING DATASET

'''

trainset = dsets.CIFAR10('./data', train= True, download= True,transform = transforms.ToTensor())
testset = dsets.CIFAR10('./data', train= False, download = True, transform = transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [238]:
len(trainset), len(testset), trainset[0][0].size(), trainset[0][1]

(50000, 10000, torch.Size([3, 32, 32]), 6)

In [239]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
CLASSES[trainset[0][1]]

'frog'

In [0]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle =True, num_workers =2)
test_loader=torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size,shuffle=True,num_workers=2)


In [0]:
'''
CREATE MODEL

'''

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.input_hidden1 = nn.Linear(32*32*3,1000)
    self.input_hidden1_dropout = nn.Dropout(0.2)
    self.hidden1_hidden2 = nn.Linear(1000,700)
    self.hidden1_hidden2_dropout = nn.Dropout(0.3)
    self.hidden2_output = nn.Linear(700,10)
    
  def forward(self, x):
    x= x.view(-1,32*32*3) #reshaping the input
    x= F.relu(self.input_hidden1(x))
    x= self.input_hidden1_dropout(x)
    x= F.tanh(self.hidden1_hidden2(x))
    x= self.hidden1_hidden2_dropout(x)
    return self.hidden2_output(x)
    

In [0]:
model = Net()
if cuda:
  model.cuda()

In [0]:
class CIFAR_MODEL:
  def __init__(self, model, lr, optimizer_value, loss_type):
    self.model = model
    self.lr = lr
    
    weightdecay = 0
    if loss_type == 'l2': #L2 regularization on loss by weightdecay
      weightdecay=0.7
      
    optimizer = ''
    if(optimizer_value == 'sgd'):
      optimizer = optim.SGD(self.model.parameters(), lr= 0.01, momentum = 0.9, weight_decay=weightdecay)
    elif(optimizer_value == 'adam'):
      optimizer = optim.Adam(self.model.parameter(), lr= 0.01, weight_decay = weightdecay)
    elif(optimizer_value == 'adagrad'):
      optimizer = optim.Adagrad(self.model.parameters(), lr= 0.01, weight_decay = weightdecay )
    elif(optimizer_value == 'adadelta'):
      optimizer = optim.Adadelta(self.model.parameters(), lr= 0.01, weight_decay = weightdecay)
    else:
      optimizer = optim.RMSprop(self.model.parameters(), lr= 0.01, weight_decay = weightdecay)
    
  def train(self, epoch, loss_type, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
      if cuda:
        data,target = data.cuda(), target.cuda()
      data, target = Variable(data), Variable(target)
      optimizer.zero_grad()
      output = model(data)
      loss = criterion(output, target)
      if loss_type == 'l1': #L1 regularisation on loss
        nn.L1Loss(loss)
      loss.backward()
      optimizer.step()
      if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
    
  def test(self, loss_vector, accuracy_vector):
    model.eval()
    test_loss, correct = 0,0
    for (data,target) in test_loader:
      if cuda:
        data, target = data.cuda(), target.cuda()
      data, target = Variable(data, volatile = True), Variable(target)
      output = model(data)
      prediction = output.data.max(1)[1]
      confusionmatrix = confusion_matrix(target,prediction)
      test_loss+= criterion(output,target).data[0]
      pred = output.data.max(1)[1]
      correct+= pred.eq(target.data).cpu().sum()
    
    test_loss/=len(test_loader)
    loss_vector.append(test_loss)
    accuracy = 100.*correct/len(test_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('Confusion_Matrix', confusionmatrix)
    print('Accuracy Score :',accuracy_score(target, prediction))
    print('\nTest set: Average loss: {:.4f}, Accuracy: ({:.0f}%)\n'.format(
        test_loss, accuracy))

In [0]:
'''
INSTANTIATE LOSS CLASS
  
'''

criterion = nn.CrossEntropyLoss()
if cuda:
  criterion = criterion.cuda()
  
def softmax(x):
  x = x - np.max(x, axis=1).reshape((-1,1))
  x = np.exp(x)
  return x / np.sum(x, axis=1).reshape((-1, 1))
  
def hingeloss(x,y):
  '''
  We were trying to impleent it like this but we were facing some error
  return (torch.clamp(torch.mm(y,x)-1,min = 0))
  '''
  return(max(0, 1-y*x))
  
def softmax_hingeloss(x,y):
  return softmax(x) + hingeloss(x,y)



In [248]:
%%time
epochs = 5
lossv,accuv = [],[]
learning_rate= 0.1
loss_type = 'l2'
cifar = CIFAR_MODEL(model, learning_rate, 'sgd', loss_type)
for epoch in range(1, epochs+1):
  cifar.train(epoch,loss_type)
  cifar.test(lossv,accuv)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.292908
Train Epoch: 1 [10000/50000 (20%)]	Loss: 2.308208
Train Epoch: 1 [20000/50000 (40%)]	Loss: 2.306726
Train Epoch: 1 [30000/50000 (60%)]	Loss: 2.328024
Train Epoch: 1 [40000/50000 (80%)]	Loss: 2.302966


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Confusion_Matrix [[ 8  0  0  0  0  0  0  0  0  0]
 [11  0  0  0  0  0  0  0  0  0]
 [11  0  0  0  0  1  0  0  0  0]
 [ 6  0  0  0  0  1  0  0  0  0]
 [11  0  0  0  0  1  0  0  0  0]
 [ 8  0  0  0  0  1  0  0  0  0]
 [16  0  0  0  0  0  0  0  0  0]
 [ 9  0  0  0  0  0  0  0  0  0]
 [12  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0]]
Accuracy Score : 0.09

Test set: Average loss: 2.3027, Accuracy: (10%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 2.314537
Train Epoch: 2 [10000/50000 (20%)]	Loss: 2.305744
Train Epoch: 2 [20000/50000 (40%)]	Loss: 2.292685
Train Epoch: 2 [30000/50000 (60%)]	Loss: 2.313194
Train Epoch: 2 [40000/50000 (80%)]	Loss: 2.300921
Confusion_Matrix [[ 8  0  0  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0]
 [12  0  0  0  0  0  0  0  0  0]
 [12  0  0  0  0  0  0  0  0  0]
 [14  0  0  0  0  1  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  0  0]
 [10  0  0  0  0  2  0  0  0  0]
 [11  0  0  0  0  1  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  0  

In [0]:
torch.save(model.state_dict(), "model_sgd.th")

In [250]:
the_model = Net()
the_model.load_state_dict(torch.load("model_sgd.th"))
print(the_model)


Net(
  (input_hidden1): Linear(in_features=3072, out_features=1000, bias=True)
  (input_hidden1_dropout): Dropout(p=0.2)
  (hidden1_hidden2): Linear(in_features=1000, out_features=700, bias=True)
  (hidden1_hidden2_dropout): Dropout(p=0.3)
  (hidden2_output): Linear(in_features=700, out_features=10, bias=True)
)


In [251]:
torch.save(model, "model_sgd.th.2")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [252]:
the_model2 = torch.load("model_sgd.th.2")
print(the_model2)

Net(
  (input_hidden1): Linear(in_features=3072, out_features=1000, bias=True)
  (input_hidden1_dropout): Dropout(p=0.2)
  (hidden1_hidden2): Linear(in_features=1000, out_features=700, bias=True)
  (hidden1_hidden2_dropout): Dropout(p=0.3)
  (hidden2_output): Linear(in_features=700, out_features=10, bias=True)
)
